In [ ]:
import sys
import tweepy
import time
import csv
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import nltk
from nltk.corpus import stopwords as swords
from wordcloud import WordCloud, STOPWORDS
import twint
import re
from textblob import TextBlob

In [ ]:
# Read your API tokens from a file
TWITTER = open("TwiTokens.bak", "r").read().splitlines()
auth = tweepy.OAuthHandler(TWITTER[6], TWITTER[7])
auth.set_access_token(TWITTER[8], TWITTER[9])

api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
# Grab info of YOUR account. This is so you can analyze your own followers.
user = api.get_user('YOUR_ACCOUNT')

In [ ]:
print(user)

In [ ]:
print(f"user: {user.screen_name}")
print(f"user ID: {user.id}")
print(f"Bio: {user.description}")
print(f"Since: {user.created_at}")
print(f"Total Tweets: {user.statuses_count}")
print(f"Following: {user.friends_count}")
print(f"Followers: {user.followers_count}")
print(f"Verified?: {user.verified}")
print(f"Last Tweet Date: {user.status.created_at}")
print(f"Last Tweet Text: {user.status.text}")

In [ ]:
followers = api.followers_ids('YOUR_ACCOUNT')
print(followers)

In [ ]:
profiles = followers[:]
print(profiles)

In [ ]:
print(f"Following: {user.friends_count}")
print(f"Followers: {user.followers_count}")
print(f"Follow ratio: {user.friends_count / user.followers_count}")

In [ ]:
profiles = followers[:] # make a copy of the list, for an original, and for one that decreases in size as it is analyzed

In [ ]:
with open('tweet_analysis.csv', 'a') as tweet_file:
    for profile in profiles:
        while True:
            try:
                print(f"Scraping {profile}")
                account = api.get_user(profile)
                csv_string = f'{account.screen_name},{account.id},{account.created_at},{account.statuses_count},{account.friends_count},{account.followers_count},{account.verified},{account.status.created_at}\n'
                print(csv_string)
                tweet_file.write(csv_string)
                tweet_file.flush()
                profiles.remove(profile)
                time.sleep(15)
            except AttributeError as a:
                print(a)
                csv_string = f'{account.screen_name},{account.id},{account.created_at},{account.statuses_count},{account.friends_count},{account.followers_count},{account.verified},NULL\n'
                print(csv_string)
                tweet_file.write(csv_string)
                tweet_file.flush()
                profiles.remove(profile)
                break
            except Exception as e:
                print("Probably a connection error. Retrying in like....15 seconds")
                time.sleep(15)
                continue
            break
tweet_file.close()    

In [ ]:
print(profiles)
print("---")
print(followers)

In [ ]:
print(len(profiles))
print(len(followers))


In [ ]:
for profile in profiles:
    while True:
        try:
            acct = api.get_user(profile)
            print(f"{acct.screen_name}: {acct.protected}")
            time.sleep(5)
        except AttributeError:
            print(f"Manually check if {acct.screen_name} is protected")
            pass
        except:
            print("Connection issue me thinks...waiting")
            time.sleep(5)
            continue
        break
            

raw_df = pd.read_csv('2021-03-01_tweet_analysis.csv')
raw_df.loc[raw_df['followng'] <= 2]
#print(raw_df)

In [ ]:
raw_df = pd.read_csv('2021-03-01_tweet_analysis.csv') 
raw_df.loc[raw_df['followers'] <= 2]

In [ ]:
active_users_df = raw_df.loc[raw_df['followers'] > 0]
print(active_users_df)

In [ ]:
print(f"Means: Following {active_users_df['following'].mean()}, Followers {active_users_df['followers'].mean()} Tweets {active_users_df['tweet_count'].mean()}")
print(f"Medians: Following {active_users_df['following'].median()}, Followers {active_users_df['followers'].median()} Tweets {active_users_df['tweet_count'].median()}")
print(f"Modes: Following {active_users_df['following'].mode()}, Followers {active_users_df['followers'].mode()} Tweets {active_users_df['tweet_count'].mode()}")
print(f"Mins: Following {active_users_df['following'].min()}, Followers {active_users_df['followers'].min()} Tweets {active_users_df['tweet_count'].min()}")
print(f"Maxes: Following {active_users_df['following'].max()}, Followers {active_users_df['followers'].max()} Tweets {active_users_df['tweet_count'].max()}")
print(f"S. Deviations: Following {active_users_df['following'].std()}, Followers {active_users_df['followers'].std()} Tweets {active_users_df['tweet_count'].std()}")

In [ ]:
active_users_df['follow_ratio'] = active_users_df["following"]/active_users_df["followers"]
print(active_users_df)

In [ ]:
active_users_df["follow_ratio"] = active_users_df["following"]/active_users_df["followers"]
active_users_df["user", "follow_ratio"]

In [ ]:
top_20_ratio = active_users_df.sort_values('follow_ratio').head(20)
top_20_follower_count = active_users_df.sort_values('followers', ascending = False).head(20)
top_20_tweet_count = active_users_df.sort_values('tweet_count', ascending = False).head(20)
influencers = []
for user in top_20_ratio['user']:
    influencers.append(user)
for user in top_20_follower_count['user']:
    influencers.append(user)
for user in top_20_tweet_count['user']:
    influencers.append(user)
print(influencers)
with open('influencers.txt', 'w+') as i_file:
    for influencer in influencers:
        i_file.write(f'{influencer}\n')
i_file.close()

In [ ]:
unique_influencers = {}
for influencer in influencers:
    if influencer not in unique_influencers:
        unique_influencers[influencer] = 1
    else:
        unique_influencers[influencer] += 1
print(unique_influencers)

In [ ]:
for influencer in unique_influencers:
    while True:
        try:
            with open(f'{influencer}.txt', 'w+') as i_file:
                for tweet in tweepy.Cursor(api.user_timeline, id=influencer).items():
                    i_file.write(f'{tweet.text}\n')
                    i_file.flush()
            i_file.close()
        except Exception as e:
            print(e)
            time.sleep(10)
            #continue
        break

In [ ]:
active_users_df = active_users_df.loc[active_users_df['tweet_count'] > 0]
print(active_users_df)

In [ ]:
verified = len(active_users_df.loc[active_users_df['verified'] == True])
unverified = len(active_users_df.loc[active_users_df['verified'] == False])
print(f"Verified: {verified}")
print(f"Unverified: {unverified}")
print(f"Verified Percentage: {verified/len(api.followers_ids(user))}")

In [ ]:
active_users_df.loc[active_users_df['follow_ratio'] <= 1.0] 

In [ ]:
#active_users_df.loc[active_users_df['following'] <= 20]
active_users_df[active_users_df['last_tweet_date'].isna()]

In [ ]:
plt.scatter(active_users_df['followers'], active_users_df['tweet_count'])
plt.show()

In [ ]:
list(active_users_df['user'])

In [ ]:
print(unique_influencers)

In [ ]:
stopwords = set(STOPWORDS)
for influencer in unique_influencers:
    word_count = 0
    rt_list = []
    word_dict = {}
    hashtags = {}
    links = []
    table = str.maketrans('', '', string.punctuation)
    with open(f'{influencer}.txt', 'r') as i_file:
        tweets = i_file.readlines()
        
        for tweet in tweets:
            tweet = tweet.lower()
            curr_string = tweet.split()
            for word in curr_string:
                if 'http:' in word or 'https:' in word:
                    links.append(word)

            if tweet.startswith('rt'):
                rt_list.append(tweet)
    
            #curr_string = tweet.split()
            for word in curr_string:
                if 'http:' in word or 'https:' in word:
                    links.append(word)
                    pass
                if word in stopwords:
                    pass
                if word.startswith('#'):
                    word = word.translate(table)
                    if word not in hashtags:
                        hashtags[word] = 1
                    else:
                        hashtags[word] += 1
                    
                word = word.translate(table)
                if word not in word_dict:
                    word_dict[word] = 1
                else:
                    word_dict[word] += 1
        full_text = " ".join(word for word in word_dict)
        try:
            word_cloud = WordCloud(stopwords=stopwords).generate(full_text)
            plt.imshow(word_cloud, interpolation='bilinear')
            plt.axis("off")
            plt.show()
        except:
            print("No words for the WordCloud apparently")
        i_file.close()
        for word in word_dict.copy():
            if word in swords.words('english'):
                del word_dict[word]
        sort_words = sorted(word_dict.items(), key=lambda x: x[1], reverse=True)
        print("---Top words---")
        for i in sort_words[:20]:
            print(i[0], i[1])
        sorted_hashtags = sorted(hashtags.items(), key=lambda x: x[1], reverse=True)
        print("---Top hashtags---")
        for i in sorted_hashtags[:20]:
            print(i[0], i[1])
        """
        try:
            print(f'{influencer}: {word_dict["cancel"]}')
            print(f'{influencer}: {word_dict["boycott"]}')
        except KeyError:
            print(f'{influencer}: No "cancel" or "boycott" found')
        except ValueError:
            pass
        """


In [ ]:
for influencer in unique_influencers:
    try:
        infl_info = api.get_user(influencer)
        print(f'{influencer}: {infl_info.description}')
    except:
        print(f'Was {influencer} suspended?')

In [ ]:
# Take each "influencer" and 
# 1. get their verified percentage
# 2. get the veerified percentage of *their* followers
# This code block is doable, but will take an unrealistic amount of time.
# It is best to skip this for now until a better algorith, shjows itself....or better metric
for influencer in influencers:
    verified = []
    unverified = []
    influencer_followers = api.followers_ids(influencer)
    for account in influencer_followers:
        acc_info = api.get_user(account)
        if acc_info.verified == True:
            verified.append(account)
        else:
            unverified.append(account)
        time.sleep(15)
    print(len(verified))
    print(len(unverified))
    verify_percent = len(verified)/len(influencer_followers)
    print(f'{influencer}: {verify_percent}')

In [ ]:
# Simple analysis of a user's bio/desription
with open("follower_bios.txt", "w+") as f_file:
    for follower in followers:
        account = api.get_user(follower)
        desc_string = f'{account.screen_name}: {account.description}'
        print(desc_string)
        f_file.write(desc_string)
        f_file.flush()
        

In [ ]:
# Sentiment analysis on the followers that were considered to be "influencers"
for influencer in unique_influencers:
    sentiments = []
    with open(f'{influencer}.txt', 'r') as test_file:
        tweets = test_file.readlines()
        for tweet in tweets:
            tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
            sent_analysis = TextBlob(tweet)
            if sent_analysis.sentiment.polarity > 0:
                sentiments.append('positive')
            elif sent_analysis.sentiment.polarity == 0:
                sentiments.append('neutral')
            else:
                sentiments.append('negative')
        print(f'---{influencer}---')
        try:
            print(f'Positive Sentiment Percentage: {sentiments.count("positive")/len(sentiments)}')
            print(f'Negative Sentiment Percentage: {sentiments.count("negative")/len(sentiments)}')
            print(f'Neutral Sentiment Percentage: {sentiments.count("neutral")/len(sentiments)}')
        except:
            print("Potential singularity here")